In [10]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
import yfinance as yf
import ta

# Disable SSL verification
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#### Machine Learning Models
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel

#### Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization, GRU, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import LeakyReLU

#### Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Fix the missing import
from sklearn.model_selection import train_test_split

class AdvancedStockPredictor:
    def __init__(self):
        self.models = {}
        self.scalers = {}
        self.results = {}
        self.lstm_scaler = None
        self.lstm_model = None
        self.currency_info = {}
        self.feature_importance = {}
        self.selected_features = {}
        self.feature_names = {}  # Store feature names
        
        self.STOCK_SYMBOL_MAP = {
            'AAPL': 'AAPL', 'APPLE': 'AAPL',
            'MSFT': 'MSFT', 'MICROSOFT': 'MSFT',
            'GOOGL': 'GOOGL', 'GOOGLE': 'GOOGL', 'ALPHABET': 'GOOGL',
            'AMZN': 'AMZN', 'AMAZON': 'AMZN',
            'TSLA': 'TSLA', 'TESLA': 'TSLA',
            'META': 'META', 'FACEBOOK': 'META',
            'NVDA': 'NVDA', 'NVIDIA': 'NVDA',
            'RELIANCE': 'RELIANCE.NS', 
            'TCS': 'TCS.NS', 
            'INFY': 'INFY.NS', 'INFOSYS': 'INFY.NS',
            'HDFC': 'HDFCBANK.NS', 'HDFCBANK': 'HDFCBANK.NS',
            'ICICI': 'ICICIBANK.NS', 'ICICIBANK': 'ICICIBANK.NS',
            'SBIN': 'SBIN.NS', 'SBI': 'SBIN.NS',
            'WIPRO': 'WIPRO.NS',
        }
        
        self.CURRENCY_MAP = {
            '.NS': 'INR ', '.KS': 'KRW ', 
            '.SW': 'CHF ', '.L': 'GBP ',
        }
        self.DEFAULT_CURRENCY = 'USD '
    
    def get_currency_symbol(self, symbol):
        for suffix, currency in self.CURRENCY_MAP.items():
            if symbol.endswith(suffix):
                return currency
        return self.DEFAULT_CURRENCY
    
    def get_correct_symbol(self, symbol_input):
        symbol_input = symbol_input.upper().strip()
        if symbol_input in self.STOCK_SYMBOL_MAP:
            return self.STOCK_SYMBOL_MAP[symbol_input]
        return symbol_input
    
    def create_technical_indicators(self, df):
        """Create technical indicators with proper error handling"""
        print("Creating technical indicators...")
        
        close = df['Close'].squeeze()
        high = df['High'].squeeze()
        low = df['Low'].squeeze()
        volume = df['Volume'].squeeze()
        
        # RSI with multiple timeframes
        for window in [7, 14, 21]:
            try:
                df[f'RSI_{window}'] = ta.momentum.rsi(close, window=window)
                df[f'RSI_{window}_SMA'] = df[f'RSI_{window}'].rolling(14).mean()
            except Exception as e:
                print(f"Warning: Could not create RSI_{window}: {e}")
        
        # Stochastic Oscillator
        try:
            df['Stoch_K'] = ta.momentum.stoch(high, low, close, window=14)
            df['Stoch_D'] = df['Stoch_K'].rolling(3).mean()
            df['Stoch_Overbought'] = (df['Stoch_K'] > 80).astype(int)
            df['Stoch_Oversold'] = (df['Stoch_K'] < 20).astype(int)
        except Exception as e:
            print(f"Warning: Could not create Stochastic: {e}")
        
        # Williams %R
        try:
            df['Williams_R'] = ta.momentum.williams_r(high, low, close, lbp=14)
        except Exception as e:
            print(f"Warning: Could not create Williams %R: {e}")
        
        # MACD (trend and momentum)
        try:
            macd = ta.trend.MACD(close)
            df['MACD'] = macd.macd()
            df['MACD_Signal'] = macd.macd_signal()
            df['MACD_Diff'] = macd.macd_diff()
            df['MACD_Cross'] = (df['MACD'] > df['MACD_Signal']).astype(int)
        except Exception as e:
            print(f"Warning: Could not create MACD: {e}")
        
        # ADX (trend strength)
        try:
            df['ADX'] = ta.trend.adx(high, low, close, window=14)
            df['ADX_Strong_Trend'] = (df['ADX'] > 25).astype(int)
        except Exception as e:
            print(f"Warning: Could not create ADX: {e}")
        
        # Bollinger Bands
        try:
            bb = ta.volatility.BollingerBands(close, window=20, window_dev=2)
            df['BB_High'] = bb.bollinger_hband()
            df['BB_Low'] = bb.bollinger_lband()
            df['BB_Mid'] = bb.bollinger_mavg()
            df['BB_Width'] = (df['BB_High'] - df['BB_Low']) / df['BB_Mid']
            df['BB_Position'] = (close - df['BB_Low']) / (df['BB_High'] - df['BB_Low'])
            df['BB_Upper_Band_Touch'] = (close >= df['BB_High']).astype(int)
            df['BB_Lower_Band_Touch'] = (close <= df['BB_Low']).astype(int)
        except Exception as e:
            print(f"Warning: Could not create Bollinger Bands: {e}")
        
        # ATR (volatility)
        try:
            df['ATR'] = ta.volatility.average_true_range(high, low, close, window=14)
            df['ATR_Pct'] = df['ATR'] / close * 100
        except Exception as e:
            print(f"Warning: Could not create ATR: {e}")
        
        # OBV (volume-price relationship)
        try:
            df['OBV'] = ta.volume.on_balance_volume(close, volume)
            df['OBV_EMA'] = df['OBV'].ewm(span=20).mean()
            df['OBV_Divergence'] = df['OBV'] - df['OBV_EMA']
        except Exception as e:
            print(f"Warning: Could not create OBV: {e}")
        
        # Advanced technical indicators
        try:
            # Commodity Channel Index
            df['CCI'] = ta.trend.cci(high, low, close, window=20)
        except Exception as e:
            print(f"Warning: Could not create CCI: {e}")
        
        try:
            # Rate of Change
            df['ROC_10'] = ta.momentum.roc(close, window=10)
            df['ROC_20'] = ta.momentum.roc(close, window=20)
        except Exception as e:
            print(f"Warning: Could not create ROC: {e}")
        
        try:
            # TRIX indicator
            df['TRIX'] = ta.trend.trix(close, window=15)
        except Exception as e:
            print(f"Warning: Could not create TRIX: {e}")
        
        try:
            # KST oscillator
            kst = ta.trend.kst(close)
            df['KST'] = kst
        except Exception as e:
            print(f"Warning: Could not create KST: {e}")
        
        try:
            # Chaikin Money Flow
            df['CMF'] = ta.volume.chaikin_money_flow(high, low, close, volume, window=20)
        except Exception as e:
            print(f"Warning: Could not create CMF: {e}")
        
        try:
            # Force Index
            df['Force_Index'] = ta.volume.force_index(close, volume, window=13)
        except Exception as e:
            print(f"Warning: Could not create Force Index: {e}")
        
        return df
    
    def create_advanced_features(self, df):
        """Create more sophisticated features for better prediction"""
        print("Creating advanced features...")
        
        # Price-based features
        df['HL_Ratio'] = df['High'] / df['Low']
        df['OC_Ratio'] = df['Close'] / df['Open']
        df['Price_Range'] = (df['High'] - df['Low']) / df['Close']
        
        # Enhanced volatility features
        for window in [5, 10, 20]:
            df[f'Rolling_Vol_{window}'] = df['Returns'].rolling(window).std()
            df[f'Rolling_Mean_{window}'] = df['Returns'].rolling(window).mean()
            df[f'Volatility_Ratio_{window}'] = df[f'Rolling_Vol_{window}'] / (df[f'Rolling_Mean_{window}'].abs() + 1e-8)
        
        # Advanced momentum indicators
        df['Momentum_1_5'] = df['Close'].pct_change(5) - df['Close'].pct_change(1)
        df['Momentum_5_20'] = df['Close'].pct_change(20) - df['Close'].pct_change(5)
        
        # Price acceleration
        df['Price_Acceleration'] = df['Returns'].diff()
        
        # Volume-price correlation
        for window in [5, 10, 20]:
            df[f'Volume_Price_Corr_{window}'] = df['Volume'].rolling(window).corr(df['Close'])
        
        # Support and resistance levels
        df['Resistance_20'] = df['High'].rolling(20).max()
        df['Support_20'] = df['Low'].rolling(20).min()
        df['Distance_to_Resistance'] = (df['Close'] - df['Resistance_20']) / df['Resistance_20']
        df['Distance_to_Support'] = (df['Close'] - df['Support_20']) / df['Support_20']
        
        # Market regime detection
        df['Volatility_Regime'] = (df['Volatility_20'] > df['Volatility_20'].rolling(50).median()).astype(int)
        df['Trend_Regime'] = (df['Close'] > df['SMA_50']).astype(int)
        
        # Price position features
        df['Price_Position_52W'] = (df['Close'] - df['Close'].rolling(252).min()) / (df['Close'].rolling(252).max() - df['Close'].rolling(252).min())
        df['Price_Position_20'] = (df['Close'] - df['Close'].rolling(20).min()) / (df['Close'].rolling(20).max() - df['Close'].rolling(20).min())
        
        # Seasonality features (day of week, month)
        df['Day_of_Week'] = df.index.dayofweek
        df['Month'] = df.index.month
        df['Week_of_Year'] = df.index.isocalendar().week
        
        # Donchian Channel
        df['Donchian_High'] = df['High'].rolling(20).max()
        df['Donchian_Low'] = df['Low'].rolling(20).min()
        df['Donchian_Mid'] = (df['Donchian_High'] + df['Donchian_Low']) / 2
        
        return df
    
    def fetch_and_prepare_data(self, symbol, years=8):
        """Fetch more historical data for better training"""
        end_date = datetime.now()
        start_date = end_date - timedelta(days=years*365)
        
        correct_symbol = self.get_correct_symbol(symbol)
        print(f"Fetching {years} years of data for {symbol} -> {correct_symbol}...")
        
        try:
            df = yf.download(correct_symbol, start=start_date, end=end_date, progress=False)
            
            if df.empty:
                ticker = yf.Ticker(correct_symbol)
                df = ticker.history(start=start_date, end=end_date)
            
            if df.empty:
                df = yf.download(correct_symbol, period="max", progress=False)
                
            if df.empty:
                raise ValueError(f"No data found for symbol: {symbol}")
            
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(1)
            
            print(f"Fetched {len(df)} records from {df.index[0].date()} to {df.index[-1].date()}")
            
            self.currency_info[symbol] = self.get_currency_symbol(correct_symbol)
            
            # BASIC FEATURE ENGINEERING
            print("Computing basic features...")
            
            # Basic price features
            df['Returns'] = df['Close'].pct_change()
            df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
            df['High_Low_Pct'] = (df['High'] - df['Low']) / df['Low'] * 100
            df['Close_Open_Pct'] = (df['Close'] - df['Open']) / df['Open'] * 100
            
            # Trend strength
            df['Trend_Strength'] = df['Close'].rolling(20).apply(
                lambda x: np.polyfit(range(len(x)), x, 1)[0], raw=True
            )
            
            # Multiple timeframe MAs - CREATE THESE FIRST
            for window in [5, 10, 20, 50, 100, 200]:
                df[f'SMA_{window}'] = df['Close'].rolling(window).mean()
                df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
                
                # Price position relative to MA
                df[f'Close_to_SMA_{window}'] = (df['Close'] - df[f'SMA_{window}']) / df[f'SMA_{window}'] * 100
            
            # NOW create MA crossovers (after EMAs are created)
            try:
                df['SMA_5_20_Cross'] = (df['SMA_5'] > df['SMA_20']).astype(int)
                df['SMA_10_50_Cross'] = (df['SMA_10'] > df['SMA_50']).astype(int)
                if 'EMA_12' in df.columns and 'EMA_26' in df.columns:
                    df['EMA_12_26_Cross'] = (df['EMA_12'] > df['EMA_26']).astype(int)
            except Exception as e:
                print(f"Warning: Could not create MA crossovers: {e}")
            
            # Volatility (critical for prediction)
            for window in [5, 10, 20, 30]:
                df[f'Volatility_{window}'] = df['Returns'].rolling(window).std() * np.sqrt(252)
                df[f'Return_Volatility_Ratio_{window}'] = df['Returns'].rolling(window).mean() / (
                    df['Returns'].rolling(window).std() + 1e-8
                )
            
            # Volume analysis
            df['Volume_SMA_20'] = df['Volume'].rolling(20).mean()
            df['Volume_Ratio'] = df['Volume'] / (df['Volume_SMA_20'] + 1)
            df['Volume_Price_Trend'] = df['Volume'] * df['Returns']
            
            # Price momentum across timeframes
            for period in [1, 3, 5, 10, 20, 30]:
                df[f'Momentum_{period}'] = df['Close'].pct_change(period) * 100
                df[f'ROC_{period}'] = ((df['Close'] - df['Close'].shift(period)) / 
                                        df['Close'].shift(period)) * 100
            
            # Create technical indicators
            df = self.create_technical_indicators(df)
            
            # ADVANCED FEATURES
            df = self.create_advanced_features(df)
            
            # Lagged features (critical for time series)
            for lag in [1, 2, 3, 5]:
                df[f'Close_Lag_{lag}'] = df['Close'].shift(lag)
                df[f'Volume_Lag_{lag}'] = df['Volume'].shift(lag)
                df[f'Returns_Lag_{lag}'] = df['Returns'].shift(lag)
            
            # Rolling statistics
            for window in [5, 10, 20]:
                df[f'Close_Rolling_Mean_{window}'] = df['Close'].rolling(window).mean()
                df[f'Close_Rolling_Std_{window}'] = df['Close'].rolling(window).std()
            
            # Clean data
            initial_len = len(df)
            df = df.replace([np.inf, -np.inf], np.nan)
            
            # Forward fill then drop remaining NaN
            df = df.ffill().dropna()
            dropped = initial_len - len(df)
            
            print(f"Dropped {dropped} rows. Final: {len(df)} records with {len(df.columns)} features")
            
            if len(df) < 300:
                raise ValueError(f"Insufficient data: {len(df)} records")
            
            return df
            
        except Exception as e:
            print(f"ERROR: {str(e)}")
            raise
    
    def feature_selection(self, X, y, method='random_forest'):
        """Select most important features to reduce noise"""
        print("Performing feature selection...")
        
        if method == 'random_forest':
            selector = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
            selector.fit(X, y)
            feature_selector = SelectFromModel(selector, prefit=True, threshold='median')
            X_selected = feature_selector.transform(X)
            selected_features = X.columns[feature_selector.get_support()]
            
            # Store feature importance
            self.feature_importance['feature_selector'] = dict(zip(
                X.columns,
                selector.feature_importances_
            ))
        else:
            # Use correlation-based selection
            corr_with_target = X.corrwith(y).abs()
            high_corr_features = corr_with_target[corr_with_target > 0.1].index
            X_selected = X[high_corr_features]
            selected_features = high_corr_features
        
        print(f"Selected {len(selected_features)} features from {len(X.columns)}")
        return X_selected, selected_features
    
    def prepare_features_for_ml(self, df, target_col='Close', n_ahead=1):
        """Prepare comprehensive feature set"""
        print(f"Preparing features (predicting {n_ahead} days ahead)...")
        
        # Exclude non-feature columns
        exclude_cols = ['Open', 'High', 'Low', 'Adj Close', 'Target']
        feature_cols = [col for col in df.columns if col not in exclude_cols]
        
        # Create target
        df_ml = df.copy()
        df_ml['Target'] = df_ml[target_col].shift(-n_ahead)
        df_ml = df_ml.dropna()
        
        X = df_ml[feature_cols]
        y = df_ml['Target']
        
        # Feature selection
        X_selected, selected_features = self.feature_selection(X, y)
        
        # Store feature names for later use
        self.feature_names['ml'] = selected_features
        
        print(f"Original features: {len(feature_cols)}, Selected: {len(selected_features)}, Samples: {len(X_selected)}")
        return X_selected, y, selected_features
    
    def prepare_lstm_data(self, df, sequence_length=60):
        """Prepare LSTM data with optimal features - IMPROVED VERSION"""
        print(f"Preparing LSTM data (sequence: {sequence_length})...")
        
        # Select most relevant features for LSTM - Focus on price and volume
        lstm_features = [
            'Close', 'Volume', 'Returns', 'Log_Returns',
            'RSI_14', 'MACD', 'MACD_Diff', 'ATR', 'ADX',
            'SMA_20', 'EMA_20', 
            'BB_Width', 'BB_Position',
            'Volatility_20', 'OBV',
            'Stoch_K', 'Williams_R',
            'HL_Ratio', 'OC_Ratio',
            'CCI', 'ROC_10'
        ]
        
        # Only use available features
        available_features = [col for col in lstm_features if col in df.columns]
        print(f"Using {len(available_features)} LSTM features")
        
        data = df[available_features].values
        
        # Use RobustScaler for LSTM to handle outliers better
        self.lstm_scaler = RobustScaler()
        scaled_data = self.lstm_scaler.fit_transform(data)
        
        X, y = [], []
        for i in range(sequence_length, len(scaled_data)):
            X.append(scaled_data[i-sequence_length:i])
            y.append(scaled_data[i, 0])  # Predict Close price
        
        X, y = np.array(X), np.array(y)
        
        print(f"LSTM sequences: {len(X)} samples, shape {X.shape}")
        return X, y, available_features
    
    def create_improved_lstm_model(self, input_shape):
        """Create a much better LSTM model for stock prediction"""
        print("Creating improved LSTM model...")
        
        model = Sequential([
            # First LSTM layer with return sequences
            LSTM(128, return_sequences=True, input_shape=input_shape,
                 kernel_regularizer=l1_l2(0.001, 0.001)),
            Dropout(0.2),
            BatchNormalization(),
            
            # Second LSTM layer
            LSTM(64, return_sequences=True,
                 kernel_regularizer=l1_l2(0.001, 0.001)),
            Dropout(0.2),
            BatchNormalization(),
            
            # Third LSTM layer
            LSTM(32, return_sequences=False,
                 kernel_regularizer=l1_l2(0.001, 0.001)),
            Dropout(0.2),
            BatchNormalization(),
            
            # Dense layers
            Dense(64, activation='relu'),
            Dropout(0.1),
            Dense(32, activation='relu'),
            Dropout(0.1),
            Dense(16, activation='relu'),
            Dense(1)  # Output layer
        ])
        
        return model
    
    def create_ensemble_model(self, X_train, y_train):
        """Create ensemble model for better performance"""
        rf = RandomForestRegressor(
            n_estimators=300,
            max_depth=25,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt',
            bootstrap=True,
            random_state=42,
            n_jobs=-1
        )
        
        gb = GradientBoostingRegressor(
            n_estimators=200,
            max_depth=8,
            learning_rate=0.05,
            subsample=0.8,
            random_state=42
        )
        
        svr = SVR(
            kernel='rbf',
            C=100,
            gamma='scale',
            epsilon=0.01,
            cache_size=1000
        )
        
        elastic = ElasticNet(
            alpha=0.01,
            l1_ratio=0.7,
            random_state=42
        )
        
        ensemble = VotingRegressor([
            ('rf', rf),
            ('gb', gb),
            ('svr', svr),
            ('elastic', elastic)
        ])
        
        return ensemble
    
    def train_models(self, X_ml, y_ml, X_lstm, y_lstm):
        """Train models with cross-validation - IMPROVED LSTM TRAINING"""
        print("\n" + "="*70)
        print("TRAINING ADVANCED MODELS")
        print("="*70)
        
        # Standard train-test split
        X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(
            X_ml, y_ml, test_size=0.15, shuffle=False
        )
        
        split_idx = int(len(X_lstm) * 0.85)
        X_train_lstm = X_lstm[:split_idx]
        X_test_lstm = X_lstm[split_idx:]
        y_train_lstm = y_lstm[:split_idx]
        y_test_lstm = y_lstm[split_idx:]
        
        print(f"ML - Train: {len(X_train_ml)}, Test: {len(X_test_ml)}")
        print(f"LSTM - Train: {len(X_train_lstm)}, Test: {len(X_test_lstm)}\n")
        
        # === ENSEMBLE MODEL ===
        print("[1/4] Training Ensemble Model...")
        ensemble_scaler = RobustScaler()
        X_train_ensemble = ensemble_scaler.fit_transform(X_train_ml)
        X_test_ensemble = ensemble_scaler.transform(X_test_ml)
        
        ensemble_model = self.create_ensemble_model(X_train_ensemble, y_train_ml)
        ensemble_model.fit(X_train_ensemble, y_train_ml)
        ensemble_pred = ensemble_model.predict(X_test_ensemble)
        
        self.models['Ensemble'] = ensemble_model
        self.scalers['Ensemble'] = ensemble_scaler
        self.results['Ensemble'] = {
            'predictions': ensemble_pred,
            'actual': y_test_ml.values
        }
        
        # === RANDOM FOREST (Optimized) ===
        print("[2/4] Training Random Forest...")
        rf_scaler = RobustScaler()
        X_train_rf = rf_scaler.fit_transform(X_train_ml)
        X_test_rf = rf_scaler.transform(X_test_ml)
        
        rf_model = RandomForestRegressor(
            n_estimators=400,
            max_depth=25,
            min_samples_split=3,
            min_samples_leaf=2,
            max_features='sqrt',
            bootstrap=True,
            oob_score=True,
            random_state=42,
            n_jobs=-1
        )
        
        rf_model.fit(X_train_rf, y_train_ml)
        rf_pred = rf_model.predict(X_test_rf)
        
        # Feature importance - use stored feature names
        if hasattr(X_ml, 'columns'):
            # If X_ml is DataFrame
            feature_names = X_ml.columns
        else:
            # If X_ml is numpy array, use stored feature names
            feature_names = self.feature_names.get('ml', [f'feature_{i}' for i in range(X_train_ml.shape[1])])
        
        self.feature_importance['Random_Forest'] = dict(zip(
            feature_names,
            rf_model.feature_importances_
        ))
        
        self.models['Random_Forest'] = rf_model
        self.scalers['Random_Forest'] = rf_scaler
        self.results['Random_Forest'] = {
            'predictions': rf_pred,
            'actual': y_test_ml.values
        }
        
        # === GRADIENT BOOSTING ===
        print("[3/4] Training Gradient Boosting...")
        gb_scaler = StandardScaler()
        X_train_gb = gb_scaler.fit_transform(X_train_ml)
        X_test_gb = gb_scaler.transform(X_test_ml)
        
        gb_model = GradientBoostingRegressor(
            n_estimators=300,
            learning_rate=0.05,
            max_depth=6,
            min_samples_split=5,
            min_samples_leaf=2,
            subsample=0.8,
            random_state=42
        )
        
        gb_model.fit(X_train_gb, y_train_ml)
        gb_pred = gb_model.predict(X_test_gb)
        
        self.models['Gradient_Boosting'] = gb_model
        self.scalers['Gradient_Boosting'] = gb_scaler
        self.results['Gradient_Boosting'] = {
            'predictions': gb_pred,
            'actual': y_test_ml.values
        }
        
        # === IMPROVED LSTM ===
        print("[4/4] Training IMPROVED LSTM...")
        
        # Create improved LSTM model
        self.lstm_model = self.create_improved_lstm_model((X_train_lstm.shape[1], X_train_lstm.shape[2]))
        
        # Use different optimizer and learning rate
        optimizer = Adam(learning_rate=0.001)
        self.lstm_model.compile(
            optimizer=optimizer, 
            loss='mse',  # Use MSE for better convergence
            metrics=['mae', 'mse']
        )
        
        # Improved callbacks
        early_stop = EarlyStopping(
            monitor='val_loss', 
            patience=15, 
            restore_best_weights=True,
            min_delta=0.001,
            verbose=1
        )
        
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=8, 
            min_lr=0.00001,
            verbose=1
        )
        
        # Model checkpoint to save best model
        checkpoint = ModelCheckpoint(
            'best_lstm_model.h5',
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            verbose=0
        )
        
        print("Training LSTM model (this may take a few minutes)...")
        history = self.lstm_model.fit(
            X_train_lstm, y_train_lstm,
            validation_data=(X_test_lstm, y_test_lstm),
            epochs=100,  # Reduced epochs but better monitoring
            batch_size=16,  # Smaller batch size for better generalization
            callbacks=[early_stop, reduce_lr, checkpoint],
            verbose=1,  # Show progress
            shuffle=False
        )
        
        # Load the best model
        self.lstm_model.load_weights('best_lstm_model.h5')
        
        # Make predictions
        lstm_pred_scaled = self.lstm_model.predict(X_test_lstm, verbose=0)
        
        # Inverse transform predictions
        n_features = X_train_lstm.shape[2]
        lstm_pred_full = np.zeros((len(lstm_pred_scaled), n_features))
        lstm_pred_full[:, 0] = lstm_pred_scaled.flatten()
        lstm_pred = self.lstm_scaler.inverse_transform(lstm_pred_full)[:, 0]
        
        # Inverse transform actual values
        y_test_full = np.zeros((len(y_test_lstm), n_features))
        y_test_full[:, 0] = y_test_lstm
        y_test_actual = self.lstm_scaler.inverse_transform(y_test_full)[:, 0]
        
        self.results['LSTM'] = {
            'predictions': lstm_pred,
            'actual': y_test_actual
        }
        
        # Display metrics
        print("\n" + "="*70)
        print("MODEL PERFORMANCE")
        print("="*70)
        print(f"{'Model':<20} {'MAE':<12} {'RMSE':<12} {'R2':<10} {'Dir_Acc':<10}")
        print("-"*70)
        
        for model in ['Ensemble', 'Random_Forest', 'Gradient_Boosting', 'LSTM']:
            self._calculate_metrics(
                self.results[model]['actual'],
                self.results[model]['predictions'],
                model
            )
        
        print("="*70 + "\n")
    
    def _calculate_metrics(self, y_true, y_pred, model_name):
        """Calculate comprehensive metrics"""
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)
        
        # Directional accuracy
        if len(y_true) > 1:
            true_dir = np.diff(y_true) > 0
            pred_dir = np.diff(y_pred) > 0
            dir_acc = np.mean(true_dir == pred_dir) * 100
        else:
            dir_acc = 0
        
        print(f"{model_name:<20} {mae:<12.2f} {rmse:<12.2f} {r2:<10.3f} {dir_acc:<10.1f}%")
        
        if model_name not in self.results:
            self.results[model_name] = {}
        
        self.results[model_name].update({
            'mae': mae, 'rmse': rmse, 'r2': r2,
            'directional_accuracy': dir_acc
        })
    
    def predict_future_7_days(self, df, feature_cols, lstm_feature_cols):
        """Generate 7-day predictions"""
        print("\nGenerating 7-day predictions...")
        
        predictions = {
            'Ensemble': [], 
            'Random_Forest': [], 
            'Gradient_Boosting': [], 
            'LSTM': []
        }
        
        last_date = df.index[-1]
        future_dates = []
        current = last_date + timedelta(days=1)
        
        while len(future_dates) < 7:
            if current.weekday() < 5:
                future_dates.append(current)
            current += timedelta(days=1)
        
        # ML predictions - convert to DataFrame for feature access
        current_data = df[feature_cols].iloc[-1:].copy()
        
        for day in range(7):
            ensemble_scaled = self.scalers['Ensemble'].transform(current_data)
            rf_scaled = self.scalers['Random_Forest'].transform(current_data)
            gb_scaled = self.scalers['Gradient_Boosting'].transform(current_data)
            
            ensemble_pred = self.models['Ensemble'].predict(ensemble_scaled)[0]
            rf_pred = self.models['Random_Forest'].predict(rf_scaled)[0]
            gb_pred = self.models['Gradient_Boosting'].predict(gb_scaled)[0]
            
            predictions['Ensemble'].append(ensemble_pred)
            predictions['Random_Forest'].append(rf_pred)
            predictions['Gradient_Boosting'].append(gb_pred)
            
            # Update current data for next prediction
            if day < 6:
                # Create new row with updated Close price
                new_row = current_data.iloc[-1:].copy()
                if 'Close' in new_row.columns:
                    new_row['Close'] = ensemble_pred
                current_data = new_row
        
        # LSTM predictions
        last_seq = df[lstm_feature_cols].values[-60:]
        current_seq = self.lstm_scaler.transform(last_seq).copy()
        
        for day in range(7):
            X_pred = current_seq.reshape(1, 60, len(lstm_feature_cols))
            pred_scaled = self.lstm_model.predict(X_pred, verbose=0)[0, 0]
            
            pred_full = np.zeros((1, len(lstm_feature_cols)))
            pred_full[0, 0] = pred_scaled
            pred_actual = self.lstm_scaler.inverse_transform(pred_full)[0, 0]
            
            predictions['LSTM'].append(pred_actual)
            
            if day < 6:
                new_row = np.zeros((1, len(lstm_feature_cols)))
                new_row[0, 0] = pred_scaled
                for i in range(1, len(lstm_feature_cols)):
                    new_row[0, i] = current_seq[-1, i]
                current_seq = np.vstack([current_seq[1:], new_row])
        
        return future_dates, predictions
    
    def create_performance_dashboard(self, symbol):
        """Create performance dashboard"""
        currency = self.currency_info.get(symbol, self.DEFAULT_CURRENCY)
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('MAE Comparison', 'R2 Score', 'Directional Accuracy', 'Model Rankings'),
            specs=[[{"type": "bar"}, {"type": "bar"}],
                   [{"type": "bar"}, {"type": "table"}]]
        )
        
        models = ['Ensemble', 'Random_Forest', 'Gradient_Boosting', 'LSTM']
        colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
        
        metrics_data = []
        for m in models:
            if m in self.results:
                metrics_data.append({
                    'Model': m,
                    'MAE': self.results[m]['mae'],
                    'R2': self.results[m]['r2'],
                    'Dir_Acc': self.results[m]['directional_accuracy']
                })
        
        metrics_df = pd.DataFrame(metrics_data)
        
        # MAE Comparison
        fig.add_trace(
            go.Bar(x=metrics_df['Model'], y=metrics_df['MAE'], 
                   marker_color=colors, name='MAE'),
            row=1, col=1
        )
        
        # R2 Score
        fig.add_trace(
            go.Bar(x=metrics_df['Model'], y=metrics_df['R2'],
                   marker_color=colors, name='R2'),
            row=1, col=2
        )
        
        # Directional Accuracy
        fig.add_trace(
            go.Bar(x=metrics_df['Model'], y=metrics_df['Dir_Acc'],
                   marker_color=colors, name='Directional Accuracy'),
            row=2, col=1
        )
        
        # Rankings table
        ranked_df = metrics_df.sort_values('R2', ascending=False)
        fig.add_trace(
            go.Table(
                header=dict(
                    values=['Rank', 'Model', 'MAE', 'R2', 'Dir_Acc'],
                    fill_color='#34495e',
                    font=dict(color='white', size=12),
                    align='left'
                ),
                cells=dict(
                    values=[
                        [f"#{i+1}" for i in range(len(ranked_df))],
                        ranked_df['Model'],
                        [f"{mae:.2f}" for mae in ranked_df['MAE']],
                        [f"{r2:.3f}" for r2 in ranked_df['R2']],
                        [f"{acc:.1f}%" for acc in ranked_df['Dir_Acc']]
                    ],
                    fill_color='lavender',
                    align='left'
                )
            ),
            row=2, col=2
        )
        
        fig.update_layout(
            title=f"{symbol} - Model Performance Dashboard",
            height=700,
            showlegend=False,
            template='plotly_white'
        )
        
        fig.show()
        return ranked_df
    
    def create_predictions_chart(self, symbol, current_price, future_dates, predictions):
        """Create predictions visualization"""
        currency = self.currency_info.get(symbol, self.DEFAULT_CURRENCY)
        
        fig = go.Figure()
        
        models = ['Ensemble', 'Random_Forest', 'Gradient_Boosting', 'LSTM']
        colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
        line_styles = ['solid', 'dash', 'dot', 'dashdot']
        
        all_dates = [future_dates[0] - timedelta(days=1)] + future_dates
        
        for i, model in enumerate(models):
            if model in predictions:
                prices = [current_price] + predictions[model]
                fig.add_trace(go.Scatter(
                    x=all_dates, y=prices,
                    mode='lines+markers',
                    name=model,
                    line=dict(color=colors[i], width=3, dash=line_styles[i]),
                    marker=dict(size=8)
                ))
        
        fig.add_hline(
            y=current_price, 
            line_dash="dash", 
            line_color="red",
            opacity=0.7,
            annotation_text=f"Current Price: {currency}{current_price:.2f}"
        )
        
        fig.update_layout(
            title=f"{symbol} - 7-Day Price Predictions",
            xaxis_title='Date',
            yaxis_title=f'Price ({currency})',
            height=500,
            template='plotly_white',
            hovermode='x unified'
        )
        
        fig.show()
    
    def create_actual_vs_predicted_chart(self, df, symbol):
        """Create actual vs predicted comparison for test data"""
        currency = self.currency_info.get(symbol, self.DEFAULT_CURRENCY)
        
        fig = go.Figure()
        
        models = ['Ensemble', 'Random_Forest', 'Gradient_Boosting', 'LSTM']
        colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']
        
        # Add actual prices
        actual_prices = self.results['Ensemble']['actual']
        dates = df.index[-len(actual_prices):]
        
        fig.add_trace(go.Scatter(
            x=dates, y=actual_prices,
            mode='lines',
            name='Actual Prices',
            line=dict(color='black', width=3)
        ))
        
        # Add predicted prices
        for i, model in enumerate(models):
            if model in self.results:
                fig.add_trace(go.Scatter(
                    x=dates, y=self.results[model]['predictions'],
                    mode='lines',
                    name=f'{model} Predictions',
                    line=dict(color=colors[i], width=2, dash='dash')
                ))
        
        fig.update_layout(
            title=f"{symbol} - Actual vs Predicted Prices (Test Set)",
            xaxis_title='Date',
            yaxis_title=f'Price ({currency})',
            height=500,
            template='plotly_white'
        )
        
        fig.show()


def main():
    """Main execution function"""
    print("="*70)
    print("ADVANCED STOCK PREDICTION SYSTEM")
    print("Models: Ensemble | Random Forest | Gradient Boosting | LSTM")
    print("="*70)
    print()
    
    # Get user input
    symbol = input("Enter stock symbol (e.g., AAPL, TSLA, TCS, RELIANCE): ").strip().upper()
    
    if not symbol:
        symbol = "AAPL"
        print(f"Using default symbol: {symbol}")
    
    try:
        # Initialize predictor
        predictor = AdvancedStockPredictor()
        
        # Fetch data
        print(f"\nFetching historical data for {symbol}...")
        df = predictor.fetch_and_prepare_data(symbol, years=8)  # More data
        
        current_price = float(df['Close'].iloc[-1])
        currency = predictor.currency_info.get(symbol, predictor.DEFAULT_CURRENCY)
        
        print(f"\nCurrent {symbol} Price: {currency}{current_price:.2f}")
        print(f"Dataset Size: {len(df)} trading days")
        print(f"Date Range: {df.index[0].date()} to {df.index[-1].date()}")
        
        # Prepare features
        X_ml, y_ml, feature_cols = predictor.prepare_features_for_ml(df)
        X_lstm, y_lstm, lstm_feature_cols = predictor.prepare_lstm_data(df)
        
        # Train models
        predictor.train_models(X_ml, y_ml, X_lstm, y_lstm)
        
        # Generate predictions
        future_dates, future_predictions = predictor.predict_future_7_days(
            df, feature_cols, lstm_feature_cols
        )
        
        # Create visualizations
        print("\nGenerating visualizations...")
        df_ranked = predictor.create_performance_dashboard(symbol)
        predictor.create_predictions_chart(symbol, current_price, future_dates, future_predictions)
        predictor.create_actual_vs_predicted_chart(df, symbol)
        
        # Print summary
        print("\n" + "="*70)
        print("PREDICTION SUMMARY")
        print("="*70)
        print(f"\nStock: {symbol}")
        print(f"Current Price: {currency}{current_price:.2f}")
        print(f"Training Period: 8 years ({len(df)} trading days)")
        
        print(f"\n7-DAY PRICE PREDICTIONS:")
        print("-"*70)
        
        best_model = df_ranked.iloc[0]['Model']
        best_r2 = df_ranked.iloc[0]['R2']
        best_dir_acc = df_ranked.iloc[0]['Dir_Acc']
        
        print(f"Best Performing Model: {best_model}")
        print(f"  - R² Score: {best_r2:.3f}")
        print(f"  - Directional Accuracy: {best_dir_acc:.1f}%")
        
        # Check LSTM performance specifically
        lstm_r2 = predictor.results['LSTM']['r2']
        lstm_dir_acc = predictor.results['LSTM']['directional_accuracy']
        
        print(f"\nLSTM Specific Performance:")
        print(f"  - R² Score: {lstm_r2:.3f}")
        print(f"  - Directional Accuracy: {lstm_dir_acc:.1f}%")
        
        if lstm_r2 > 0.7:
            print("✅ LSTM ACHIEVED TARGET ACCURACY (>70% R²)!")
        else:
            print("⚠️  LSTM needs improvement")
        
        if best_r2 > 0.8 and best_dir_acc > 75:
            print("✅ EXCELLENT MODEL ACCURACY ACHIEVED!")
        elif best_r2 > 0.7 and best_dir_acc > 65:
            print("✅ GOOD MODEL ACCURACY ACHIEVED!")
        else:
            print("⚠️  Model accuracy needs improvement")
        
        print(f"\nDetailed Predictions:")
        print("-" * 50)
        
        for i, date in enumerate(future_dates):
            ensemble_pred = future_predictions['Ensemble'][i]
            rf_pred = future_predictions['Random_Forest'][i]
            gb_pred = future_predictions['Gradient_Boosting'][i]
            lstm_pred = future_predictions['LSTM'][i]
            
            # Calculate ensemble average (weighted by R2)
            weights = {
                'Ensemble': max(0, predictor.results['Ensemble']['r2']),
                'Random_Forest': max(0, predictor.results['Random_Forest']['r2']),
                'Gradient_Boosting': max(0, predictor.results['Gradient_Boosting']['r2']),
                'LSTM': max(0, predictor.results['LSTM']['r2'])
            }
            total_weight = sum(weights.values())
            
            if total_weight > 0:
                weighted_avg = (
                    weights['Ensemble'] * ensemble_pred +
                    weights['Random_Forest'] * rf_pred +
                    weights['Gradient_Boosting'] * gb_pred +
                    weights['LSTM'] * lstm_pred
                ) / total_weight
            else:
                weighted_avg = (ensemble_pred + rf_pred + gb_pred + lstm_pred) / 4
            
            avg_change = ((weighted_avg - current_price) / current_price) * 100
            
            print(f"{date.strftime('%Y-%m-%d (%a)')}:")
            print(f"  Ensemble:        {currency}{ensemble_pred:>8.2f}  ({((ensemble_pred-current_price)/current_price*100):+6.2f}%)")
            print(f"  Random Forest:   {currency}{rf_pred:>8.2f}  ({((rf_pred-current_price)/current_price*100):+6.2f}%)")
            print(f"  Gradient Boost:  {currency}{gb_pred:>8.2f}  ({((gb_pred-current_price)/current_price*100):+6.2f}%)")
            print(f"  LSTM:            {currency}{lstm_pred:>8.2f}  ({((lstm_pred-current_price)/current_price*100):+6.2f}%)")
            print(f"  Weighted Avg:    {currency}{weighted_avg:>8.2f}  ({avg_change:+6.2f}%)")
            print()
        
        # Final recommendations
        print("="*70)
        print("ANALYSIS COMPLETE")
        print("="*70)
        print(f"\nBest Model: {best_model}")
        print(f"Model Performance:")
        print(f"  - R-Squared: {best_r2:.3f}")
        print(f"  - Directional Accuracy: {best_dir_acc:.1f}%")
        print(f"  - MAE: {currency}{predictor.results[best_model]['mae']:.2f}")
        
        print(f"\nLSTM Performance:")
        print(f"  - R-Squared: {lstm_r2:.3f}")
        print(f"  - Directional Accuracy: {lstm_dir_acc:.1f}%")
        print(f"  - MAE: {currency}{predictor.results['LSTM']['mae']:.2f}")
        
        if lstm_r2 > 0.7:
            print("\n🎯 EXCELLENT: LSTM achieved target accuracy (>70% R²)!")
        elif lstm_r2 > 0.6:
            print("\n✅ GOOD: LSTM shows decent predictive power!")
        else:
            print("\n⚠️  MODERATE: LSTM needs further optimization")
        
        print("\nNOTE: These predictions are for educational purposes only.")
        print("Always conduct thorough research before making investment decisions.")
        
    except Exception as e:
        print(f"\nERROR: {str(e)}")
        print("\nTroubleshooting Guide:")
        print("1. Verify the stock symbol is correct")
        print("2. For Indian stocks, use: TCS, INFY, RELIANCE, HDFC, etc.")
        print("3. For US stocks, use: AAPL, TSLA, MSFT, GOOGL, etc.")
        print("4. Check your internet connection")
        print("5. Some stocks may have limited historical data")
        
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

ADVANCED STOCK PREDICTION SYSTEM
Models: Ensemble | Random Forest | Gradient Boosting | LSTM


Fetching historical data for TCS...
Fetching 8 years of data for TCS -> TCS.NS...
Fetched 1977 records from 2017-10-09 to 2025-10-06
Computing basic features...
Creating technical indicators...
Creating advanced features...
Dropped 251 rows. Final: 1726 records with 137 features

Current TCS Price: INR 2988.40
Dataset Size: 1726 trading days
Date Range: 2018-10-12 to 2025-10-06
Preparing features (predicting 1 days ahead)...
Performing feature selection...
Selected 67 features from 134
Original features: 134, Selected: 67, Samples: 1725
Preparing LSTM data (sequence: 60)...
Using 21 LSTM features
LSTM sequences: 1666 samples, shape (1666, 60, 21)

TRAINING ADVANCED MODELS
ML - Train: 1466, Test: 259
LSTM - Train: 1416, Test: 250

[1/4] Training Ensemble Model...
[2/4] Training Random Forest...
[3/4] Training Gradient Boosting...
[4/4] Training IMPROVED LSTM...
Creating improved LSTM model...


89/89 ━━━━━━━━━━━━━━━━━━━━ 62s 297ms/step - loss: 3.4906 - mae: 0.3474 - mse: 0.1871 - val_loss: 3.0644 - val_mae: 0.2963 - val_mse: 0.1388 - learning_rate: 0.0010
Epoch 2/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - loss: 3.4452 - mae: 0.7303 - mse: 0.6949

89/89 ━━━━━━━━━━━━━━━━━━━━ 23s 251ms/step - loss: 2.9520 - mae: 0.5044 - mse: 0.3773 - val_loss: 2.4026 - val_mae: 0.3517 - val_mse: 0.1944 - learning_rate: 0.0010
Epoch 3/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - loss: 2.5754 - mae: 0.6450 - mse: 0.5260

89/89 ━━━━━━━━━━━━━━━━━━━━ 38s 221ms/step - loss: 2.2117 - mae: 0.4664 - mse: 0.3188 - val_loss: 1.8274 - val_mae: 0.4176 - val_mse: 0.2473 - learning_rate: 0.0010
Epoch 4/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - loss: 1.9385 - mae: 0.6171 - mse: 0.4824

89/89 ━━━━━━━━━━━━━━━━━━━━ 22s 246ms/step - loss: 1.6358 - mae: 0.4514 - mse: 0.2998 - val_loss: 1.3867 - val_mae: 0.4354 - val_mse: 0.2772 - learning_rate: 0.0010
Epoch 5/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - loss: 1.4847 - mae: 0.6016 - mse: 0.4554

...

89/89 ━━━━━━━━━━━━━━━━━━━━ 12s 137ms/step - loss: 0.3938 - mae: 0.4395 - mse: 0.2797 - val_loss: 0.3761 - val_mae: 0.4494 - val_mse: 0.2773 - learning_rate: 0.0010
Epoch 16/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 13s 149ms/step - loss: 0.3811 - mae: 0.4411 - mse: 0.2814 - val_loss: 0.4042 - val_mae: 0.4967 - val_mse: 0.3177 - learning_rate: 0.0010
Epoch 17/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 157ms/step - loss: 0.3602 - mae: 0.4404 - mse: 0.2787 - val_loss: 0.3821 - val_mae: 0.4765 - val_mse: 0.3079 - learning_rate: 0.0010
Epoch 18/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 11s 128ms/step - loss: 0.3664 - mae: 0.4388 - mse: 0.2784 - val_loss: 0.3777 - val_mae: 0.4803 - val_mse: 0.3090 - learning_rate: 0.0010
Epoch 19/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 0.4845 - mae: 0.5755 - mse: 0.4156

89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 160ms/step - loss: 0.3399 - mae: 0.4378 - mse: 0.2784 - val_loss: 0.3646 - val_mae: 0.4759 - val_mse: 0.3063 - learning_rate: 0.0010
Epoch 20/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 13s 146ms/step - loss: 0.3413 - mae: 0.4378 - mse: 0.2780 - val_loss: 0.3673 - val_mae: 0.4792 - val_mse: 0.3101 - learning_rate: 0.0010
Epoch 21/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.4585 - mae: 0.5692 - mse: 0.4069

89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 154ms/step - loss: 0.3184 - mae: 0.4366 - mse: 0.2770 - val_loss: 0.3429 - val_mae: 0.4749 - val_mse: 0.3054 - learning_rate: 0.0010
Epoch 22/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 16s 179ms/step - loss: 0.3268 - mae: 0.4376 - mse: 0.2756 - val_loss: 0.3684 - val_mae: 0.5030 - val_mse: 0.3332 - learning_rate: 0.0010
Epoch 23/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 17s 188ms/step - loss: 0.3045 - mae: 0.4368 - mse: 0.2748 - val_loss: 0.3641 - val_mae: 0.5052 - val_mse: 0.3356 - learning_rate: 0.0010
Epoch 24/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 18s 201ms/step - loss: 0.3035 - mae: 0.4366 - mse: 0.2741 - val_loss: 0.4113 - val_mae: 0.5477 - val_mse: 0.3803 - learning_rate: 0.0010
Epoch 25/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 16s 180ms/step - loss: 0.3099 - mae: 0.4391 - mse: 0.2755 - val_loss: 0.3607 - val_mae: 0.5054 - val_mse: 0.3358 - learning_rate: 0.0010
Epoch 26/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 17s 192ms/step - loss: 0.3049 - mae: 0.4378 - mse: 0.2738 - val_loss: 0.3440 - val_mae: 

89/89 ━━━━━━━━━━━━━━━━━━━━ 25s 279ms/step - loss: 0.2966 - mae: 0.4373 - mse: 0.2726 - val_loss: 0.3420 - val_mae: 0.4909 - val_mse: 0.3213 - learning_rate: 0.0010
Epoch 29/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 39s 260ms/step - loss: 0.3011 - mae: 0.4391 - mse: 0.2741 - val_loss: 0.3472 - val_mae: 0.4961 - val_mse: 0.3264 - learning_rate: 0.0010
Epoch 30/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 24s 265ms/step - loss: 0.2940 - mae: 0.4379 - mse: 0.2730 - val_loss: 0.3971 - val_mae: 0.5389 - val_mse: 0.3709 - learning_rate: 0.0010
Epoch 31/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 26s 291ms/step - loss: 0.2963 - mae: 0.4378 - mse: 0.2727 - val_loss: 0.3471 - val_mae: 0.4960 - val_mse: 0.3263 - learning_rate: 0.0010
Epoch 32/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 27s 299ms/step - loss: 0.2944 - mae: 0.4372 - mse: 0.2730 - val_loss: 0.3464 - val_mae: 0.4960 - val_mse: 0.3264 - learning_rate: 0.0010
Epoch 33/100
89/89 ━━━━━━━━━━━━━━━━━━━━ 42s 302ms/step - loss: 0.2995 - mae: 0.4374 - mse: 0.2734 - val_loss: 0.3559 - val_mae: 


PREDICTION SUMMARY

Stock: TCS
Current Price: INR 2988.40
Training Period: 8 years (1726 trading days)

7-DAY PRICE PREDICTIONS:
----------------------------------------------------------------------
Best Performing Model: Gradient_Boosting
  - R² Score: 0.949
  - Directional Accuracy: 51.2%

LSTM Specific Performance:
  - R² Score: -3.001
  - Directional Accuracy: 51.8%
⚠️  LSTM needs improvement
⚠️  Model accuracy needs improvement

Detailed Predictions:
--------------------------------------------------
2025-10-07 (Tue):
  Ensemble:        INR  2992.05  ( +0.12%)
  Random Forest:   INR  2980.41  ( -0.27%)
  Gradient Boost:  INR  3007.47  ( +0.64%)
  LSTM:            INR  2884.92  ( -3.46%)
  Weighted Avg:    INR  2993.46  ( +0.17%)

2025-10-08 (Wed):
  Ensemble:        INR  2992.24  ( +0.13%)
  Random Forest:   INR  2981.73  ( -0.22%)
  Gradient Boost:  INR  3012.80  ( +0.82%)
  LSTM:            INR  2884.91  ( -3.46%)
  Weighted Avg:    INR  2995.84  ( +0.25%)

2025-10-09 (Thu):
 